In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/train.csv')
df_train.head()

In [ ]:
df_train.describe()

In [ ]:
import holidays

contry_holidays = {
    'Finland': holidays.Finland(),
    'Norway': holidays.Norway(),
    'Sweden': holidays.SWE()
}

My first think was 'I buy a hat on sunny seasons and a mug on cold season'.

In [ ]:
df_train.info();

In [ ]:
df_train['date'] = pd.to_datetime(df_train['date'])
df_train.info()

In [ ]:
df_train.shape

In [ ]:
df_train['country'].value_counts()

:thinking: all Europeans country

# I think it's a good idea know more about EU seasons
*I'm from Brazil, it's totaly different here*

![seasons](https://hi-static.z-dn.net/files/dfa/329fad119d9efa1f4cd463536e471f8d.jpg)

> In Brazil, Janueary its summer time

In [ ]:
winter = (1,2,3)
spring = (4,5,6)
summer = (7,8,9)
autum  = (10,11,12)

season_dict = {}

for month in range(1,13):
    if month < 4:
        season_dict[month] = 'winter'
        continue
    if month >= 4 and month < 7:
        season_dict[month] = 'spring'
        continue
    if month >= 7 and month < 10:
        season_dict[month] = 'summer'
        continue
    season_dict[month] = 'autum'

In [ ]:
season_dict

In [ ]:
#contry_holidays
def set_new_features(df):
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['day'] = df['date'].dt.day
    df['weekday'] = df['date'].dt.weekday
    df['holiday'] = df.apply(lambda x: x['date'] in contry_holidays[x['country']], axis=1)
    df['weekdend'] =  df['weekday'].apply(lambda x: x in (4,5,6))
    df['season'] = df.apply(lambda x: season_dict[x['month']], axis=1)
    df = df.drop('date', axis=1)
    
    return df;


In [ ]:
df_train = set_new_features(df_train)
df_train

In [ ]:
df_train.head()

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

In [ ]:
plt.hist(x=df_train['num_sold']);

In [ ]:
px.box(df_train['num_sold'])

I think i gonna try with all data, or apply IQR range to cut off the outliers

In [ ]:
df_train.head()

In [ ]:
tree_graf = px.treemap(df_train, path=['product'])
tree_graf.show()

This dataset is cleary balanced

In [ ]:
df_train = df_train.drop('row_id', axis=1)

In [ ]:
y = df_train.pop('num_sold').values

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(df_train, y, train_size=0.25, random_state=0)

In [ ]:
X_train.shape

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
import numpy as np

preprocessor = make_column_transformer(
        (StandardScaler(), make_column_selector(dtype_include=np.number)),
        (OneHotEncoder(sparse=False, handle_unknown='ignore'), make_column_selector(dtype_include=object)),
        remainder='passthrough'
)

In [ ]:
X_train_p = preprocessor.fit_transform(X_train)
X_valid_p = preprocessor.transform(X_valid)
y_train_l = np.log(y_train)
y_valid_l = np.log(y_valid)

In [ ]:
df_p = pd.DataFrame(X_train_p)
df_p['x'] = y_train
corr = df_p.corr()
corr.style.background_gradient(cmap='coolwarm')

we've got some good correlations here.

In [ ]:
# from sklearn.linear_model import LinearRegression

# reg = LinearRegression().fit(X_train_p, y_train_l)
# y_prediction = reg.predict(X_valid_p)
# y_prediction

In [ ]:
input_shape = [X_train_p.shape[1]]

from tensorflow.keras import callbacks
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks

early_stopping = callbacks.EarlyStopping(
    min_delta=0.00001,
    patience=20,
    restore_best_weights=True,
)

model = keras.Sequential([
    layers.Dense(2024, activation='relu', input_shape=input_shape),
    layers.GaussianDropout(0.8),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(1),
])
model.compile(
    optimizer='adam',
    loss='MSLE',
)
history = model.fit(
    X_train_p, y_train_l,
    validation_data=(X_valid_p, y_valid_l),
    batch_size=45,
    epochs=500,
    callbacks=[early_stopping],
    verbose=0
)
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()));


In [ ]:
y_prediction = model.predict(X_valid_p)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = mean_squared_error(np.exp(y_valid_l), np.exp(y_prediction) , squared=True)
mse = mean_absolute_error(np.exp(y_valid_l), np.exp(y_prediction))
print(f'RMSE: {rmse}')
print(f'MAE: {mse}')

In [ ]:
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/test.csv', parse_dates=['date'])
df_test

In [ ]:
df_test = set_new_features(df_test)
df_test

In [ ]:
rows_test = df_test.pop('row_id')
df_test

In [ ]:
df_test_p = preprocessor.transform(df_test)

In [ ]:
df_test_p

In [ ]:
pred = model.predict(df_test_p)
pred

In [ ]:
result_df = pd.DataFrame({'row_id': rows_test, 'num_sold': np.exp(pred).flatten()})
result_df['num_sold'] = result_df['num_sold'].round()
result_df.to_csv('submission.csv', index=False)
result_df